<a href="https://colab.research.google.com/github/LatiefDataVisionary/data-science-capstone-project-college/blob/main/04_programmatic_labeling_bilingual_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import kagglehub
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

# # Download latest version
# path = kagglehub.dataset_download("ucupsedaya/gojek-app-reviews-bahasa-indonesia")

# print("Path to dataset files:", path)

# df = pd.read_csv(path + '\GojekAppReviewV4.0.0-V4.9.3_Cleaned.csv', encoding='unicode_escape')
# df = pd.read_csv("D:\Code\Capstone\dataset_komentar_instagram_cyberbullying.csv")
df = pd.read_csv('https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/processed/reviews_cleaned_id.csv')
df = df.dropna()

In [ ]:
df = df
df

In [ ]:
# df = df.rename(columns={"Sentiment":'sentiment', "Instagram Comment Text":"text"})
df = df.rename(columns={"Sentiment":'sentiment', "cleaned_content":"text"})

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

pretrained= "mdhugol/indonesia-bert-sentiment-classification"

model = AutoModelForSequenceClassification.from_pretrained(pretrained)
tokenizer = AutoTokenizer.from_pretrained(pretrained)

sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

label_index = {'LABEL_0': 'positive', 'LABEL_1': 'neutral', 'LABEL_2': 'negative'}

pos_text = "aplikasinya lumayan cepet"
neg_text = "Dasar anak sialan!! Kurang ajar!!"

result = sentiment_analysis(pos_text)
status = label_index[result[0]['label']]
score = result[0]['score']
print(f'Text: {pos_text} | Label : {status} ({score * 100:.3f}%)')

result = sentiment_analysis(neg_text)
status = label_index[result[0]['label']]
score = result[0]['score']
print(f'Text: {neg_text} | Label : {status} ({score * 100:.3f}%)')

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class SentimentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        text = str(self.dataframe.loc[index, 'text'])
        # sentiment = self.dataframe.loc[index, 'sentiment']

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length', # Changed from pad_to_max_length=True
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        # Map sentiment string to numerical label
        # sentiment_map = {'positive': 0, 'neutral': 1, 'negative': 2}
        # label = sentiment_map[sentiment]

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            # 'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# from sklearn.model_selection import train_test_split

# # Assuming 'df' is your pandas DataFrame with 'text' and 'sentiment' columns
# # and 'tokenizer' is your loaded tokenizer
# # and 'max_len' is the maximum sequence length

# # Split data into training and validation sets
# train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# # Create Dataset instances
# train_dataset = SentimentDataset(train_df.reset_index(drop=True), tokenizer, max_len=128) # Adjust max_len as needed
# val_dataset = SentimentDataset(val_df.reset_index(drop=True), tokenizer, max_len=128)   # Adjust max_len as needed

# # Create DataLoader instances
# BATCH_SIZE = 16 # Adjust batch size as needed
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# print("Train DataLoader created with", len(train_dataloader), "batches")
# print("Validation DataLoader created with", len(val_dataloader), "batches")

## **Labeling**

In [ ]:
test_dataset = SentimentDataset(df.reset_index(drop=True), tokenizer, max_len=128) # Adjust max_len as needed
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [ ]:
model_ft = model
# num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
# model_ft.fc = nn.Linear(num_ftrs, 2)
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
phase = "test"
running_loss = 0
running_corrects = 0
n = len(df)
y_list = []

# Use DataLoader for test_loader


for batch in test_dataloader:
    inputs = {
        'input_ids': batch['input_ids'].to(device),
        'attention_mask': batch['attention_mask'].to(device)
    }
    # labels = batch['labels'].to(device)
    # print(inputs)

    # zero the parameter gradients
    # optimizer.zero_grad()

    # forward
    # track history if only in train
    with torch.no_grad(): # Use torch.no_grad() for evaluation phase
        # Access the logits explicitly
        outputs = model(**inputs).logits
        # print('test')
        _, preds = torch.max(outputs, 1)
        # print(preds)
        y_list.append(preds)
        # loss = criterion(outputs, labels) # No labels in test set

    # statistics
    # running_loss += loss.item() * labels.size(0) # No labels in test set
    # running_corrects += torch.sum(preds == labels.data) # No labels in test set

y_val = torch.cat(y_list, 0)

# Calculate evaluation metrics (e.g., accuracy if labels were available)
# Since no labels are available, we can just print the predictions or analyze y_val
print("Predictions concatenated:", y_val)

In [ ]:
# Convert the predictions to a list
y_pred_list = y_val.cpu().numpy().tolist()

# Map numerical predictions to sentiment strings using label_index
predicted_sentiments = [label_index[f'LABEL_{pred}'] for pred in y_pred_list]

# Add the predicted sentiments as a new column to the DataFrame
df['predicted_sentiment'] = predicted_sentiments

# Display the updated DataFrame
display(df)

## **Transfer learning/ fine tuning model**

In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                    dataloader = train_dataloader
                else:
                    model.eval()   # Set model to evaluate mode
                    dataloader = val_dataloader

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for batch in dataloader:
                    inputs = {
                        'input_ids': batch['input_ids'].to(device),
                        'attention_mask': batch['attention_mask'].to(device)
                    }
                    labels = batch['labels'].to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(**inputs).logits
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * labels.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / len(dataloader.dataset)
                epoch_acc = running_corrects.double() / len(dataloader.dataset)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [ ]:
for param in model_ft.parameters():
    param.requires_grad = False
# Unfreeze classifier
for p in model.classifier.parameters():
    p.requires_grad = True
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

In [ ]:
model_ft

In [ ]:
model_ft.classifier

In [ ]:
for param in model_ft.parameters():
    param.requires_grad = False
    # print(param)